In [1]:
! pip install datasets transformers accelerate peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
qa_df = pd.read_csv("/content/data2.csv",error_bad_lines=False, engine="python")

<ipython-input-2-bb7a7916bca6>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  qa_df = pd.read_csv("/content/data2.csv",error_bad_lines=False, engine="python")


In [3]:
qa_df.head()

,Abstract,Question,Answer,Label,TemporalAspect
0,"Wallace, 38, called Gastonia home from the age...",How often does Brewer talk about the legal sys...,pushkin liked to speak at high noon usually,yes,Frequency
1,He tried and tried to please every person who ...,What time of day was it?,5 times a century,yes,Frequency
2,"One day, as Mary was picking the blueberries, ...",Can you always see them from the same location?,3:00 AM,yes,Typical Time
3,"The exhibition's range, while very varied, rem...",Is Hall still serving?,when asked,yes,Typical Time
4,Pushkin published his first poem at the age of...,How many times a week does the average scienti...,about once a century,yes,Event Duration


In [4]:
# Remove rows with any NaN values
qa_df.dropna(axis=0, how='any', inplace=True)

In [5]:
qa_df["source"] = qa_df.apply(lambda x: "context-question-answering: context: " + x["Abstract"] + " question: " + x["Question"] + " answer: " + x["Answer"], axis=1)
qa_df["target"] = qa_df["Label"]

In [6]:
qa_df

,Abstract,Question,Answer,Label,TemporalAspect,source,target
0,"Wallace, 38, called Gastonia home from the age...",How often does Brewer talk about the legal sys...,pushkin liked to speak at high noon usually,yes,Frequency,"context-question-answering: context: Wallace, ...",yes
1,He tried and tried to please every person who ...,What time of day was it?,5 times a century,yes,Frequency,context-question-answering: context: He tried ...,yes
2,"One day, as Mary was picking the blueberries, ...",Can you always see them from the same location?,3:00 AM,yes,Typical Time,"context-question-answering: context: One day, ...",yes
3,"The exhibition's range, while very varied, rem...",Is Hall still serving?,when asked,yes,Typical Time,context-question-answering: context: The exhib...,yes
4,Pushkin published his first poem at the age of...,How many times a week does the average scienti...,about once a century,yes,Event Duration,context-question-answering: context: Pushkin p...,yes
...,...,...,...,...,...,...,...
4995,"Like all energy, gravitational potential energ...",How long does the dermis take to heal after a ...,for 30 weeks,no,Frequency,context-question-answering: context: Like all ...,no
4996,Deborah likes to add some sugar with the lettu...,Is the French state still being centralized?,twice a decade,no,Event Ordering,context-question-answering: context: Deborah l...,no
4997,"A. DeConcini Federal Courthouse, 405 W. Congre...",When did Frank's father become an alcoholic?,yes they do it every month for fun,no,Frequency,context-question-answering: context: A. DeConc...,no
4998,"Search ""Emiliano Malaquez"" and you'll find he'...",Was he still being introduced an hour later?,about once a century,no,Frequency,"context-question-answering: context: Search ""E...",no


In [7]:
# Selecting only the "source" and "target" columns
final_df = qa_df[["source", "target"]]

# Creating a new DataFrame with only "source" and "target" columns
final_df = pd.DataFrame(final_df)

final_df

,source,target
0,"context-question-answering: context: Wallace, ...",yes
1,context-question-answering: context: He tried ...,yes
2,"context-question-answering: context: One day, ...",yes
3,context-question-answering: context: The exhib...,yes
4,context-question-answering: context: Pushkin p...,yes
...,...,...
4995,context-question-answering: context: Like all ...,no
4996,context-question-answering: context: Deborah l...,no
4997,context-question-answering: context: A. DeConc...,no
4998,"context-question-answering: context: Search ""E...",no


In [8]:
type(qa_df)

pandas.core.frame.DataFrame

In [9]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from sklearn.model_selection import train_test_split

In [10]:
train_df, test_df = train_test_split(final_df, test_size=0.1, random_state=0, shuffle=True)
train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)

In [11]:
model_id="google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
def preprocess_function(sample,padding="max_length"):
    model_inputs = tokenizer(sample["source"], max_length=256, padding=padding, truncation=True)
    labels = tokenizer(sample["target"], max_length=128, padding=padding, truncation=True)
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
train_tokenized_dataset = train_data.map(preprocess_function, batched=True, remove_columns=train_data.column_names)
test_tokenized_dataset = test_data.map(preprocess_function, batched=True, remove_columns=test_data.column_names)
print(f"Keys of tokenized dataset: {list(train_tokenized_dataset.features)}")

Map:   0%|          | 0/4499 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [14]:
lora_config = LoraConfig(
 r=8,
 lora_alpha=16,
 lora_dropout=0.1,
 bias="none",
 task_type="SEQ_2_SEQ_LM",
 target_modules=["q", "v"]
)

In [15]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [16]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 785,509,376 || trainable%: 0.30035236651331837


In [17]:
model

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 1024)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 1024)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1024, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=1024, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
         

In [18]:
label_pad_token_id = -100
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [19]:
from huggingface_hub import notebook_login
notebook_login()

In [20]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.2 MB/s eta 0:00:00


In [25]:
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 11.8 MB/s eta 0:00:00


In [28]:
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning


  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-ewwwzfxd
  Running command git clone --filter=blob:none --quiet https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-ewwwzfxd
  Resolved https://github.com/PyTorchLightning/pytorch-lightning to commit 858803236e02653fadd25f2fe3c01ad1eadb5311
  Running command git submodule update --init --recursive -q
  Encountered 22 file(s) that should have been pointers, but weren't:
        .notebooks/course_UvA-DL/01-introduction-to-pytorch.ipynb
        .notebooks/course_UvA-DL/02-activation-functions.ipynb
        .notebooks/course_UvA-DL/03-initialization-and-optimization.ipynb
        .notebooks/course_UvA-DL/04-inception-resnet-densenet.ipynb
        .notebooks/course_UvA-DL/05-transformers-and-MH-attention.ipynb
        .notebooks/course_UvA-DL/06-graph-neural-networks.ipynb
        .notebooks/course_UvA-DL/07-deep-energy-based-generative-models.ipynb
        .notebooks/course_UvA-DL/08-

In [33]:
import optuna
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch

# Define your model, data_collator, train_tokenized_dataset, and test_tokenized_dataset beforehand

def objective(trial):
    num_epochs = trial.suggest_int('num_epochs', 3, 10)  # Reducing the range for num_epochs
    batch_size = trial.suggest_categorical('batch_size', [4, 8])  # Limiting batch sizes
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 5e-4, log=True)  # Adjusting learning rate range

    output_dir = "flant5-large-answ"

    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,
        learning_rate=learning_rate,
        num_train_epochs=num_epochs,
        logging_dir=f"{output_dir}/logs",
        logging_strategy="epoch",
        save_strategy="epoch",
        report_to="tensorboard",
        push_to_hub=True
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_tokenized_dataset,
        eval_dataset=test_tokenized_dataset
    )

    try:
        trainer.train()
        result = trainer.evaluate()
        return result['eval_loss']
    except RuntimeError as e:
        print(f"Trial {trial.number} failed with CUDA out-of-memory error. Trying with different parameters.")
        torch.cuda.empty_cache()
        return float('inf')


# Strategy: Optimize Fewer Trials Initially
initial_trials = 3  # Start with a smaller number of trials

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=initial_trials)

best_params = study.best_params

[I 2023-12-22 15:25:05,199] A new study created in memory with name: no-name-1c331ba6-2233-4a79-9319-2f23d5f76452


Step,Training Loss
1125,0.350400
2250,0.356100
3375,0.352500
4500,0.350900


Step,Training Loss
1125,0.350400
2250,0.356100
3375,0.352500
4500,0.350900
5625,0.347400


[I 2023-12-22 16:53:36,785] Trial 0 finished with value: 0.3396131694316864 and parameters: {'num_epochs': 5, 'batch_size': 4, 'learning_rate': 0.00041755422061748147}. Best is trial 0 with value: 0.3396131694316864.


Step,Training Loss
1125,0.332300
2250,0.329000
3375,0.326400
4500,0.324700
5625,0.335500
6750,0.342500
7875,0.345500
9000,0.342300


[I 2023-12-22 19:14:25,297] Trial 1 finished with value: 0.33972403407096863 and parameters: {'num_epochs': 8, 'batch_size': 4, 'learning_rate': 1.2562851952725805e-05}. Best is trial 0 with value: 0.3396131694316864.


Step,Training Loss
1125,0.327500
2250,0.328500
3375,0.326000
4500,0.328000
5625,0.334200
6750,0.336900


[I 2023-12-22 21:00:10,931] Trial 2 finished with value: 0.35782745480537415 and parameters: {'num_epochs': 6, 'batch_size': 4, 'learning_rate': 0.00022326618038724265}. Best is trial 0 with value: 0.3396131694316864.


In [34]:
import json

# Save best_params to a JSON file
with open('best_hyperparameters(T5).json', 'w') as file:
    json.dump(best_params, file)

In [31]:
import optuna
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch

# Define your model, data_collator, train_tokenized_dataset, and test_tokenized_dataset beforehand

def objective(trial):
    num_epochs = trial.suggest_int('num_epochs', 3, 8)  # Reducing the range for num_epochs
    batch_size = trial.suggest_categorical('batch_size', [4, 8])  # Limiting batch sizes
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 5e-4, log=True)  # Adjusting learning rate range

    output_dir = "flant5-large-answ"

    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,
        learning_rate=learning_rate,
        num_train_epochs=num_epochs,
        logging_dir=f"{output_dir}/logs",
        logging_strategy="epoch",
        save_strategy="epoch",
        report_to="tensorboard",
        push_to_hub=True
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_tokenized_dataset,
        eval_dataset=test_tokenized_dataset
    )

    try:
        trainer.train()
        result = trainer.evaluate()
        return result['eval_loss']
    except RuntimeError as e:
        print(f"Trial {trial.number} failed with CUDA out-of-memory error. Trying with different parameters.")
        torch.cuda.empty_cache()
        return float('inf')


# Strategy: Optimize Fewer Trials Initially
initial_trials = 3  # Start with a smaller number of trials

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=initial_trials)

best_params = study.best_params

[I 2023-12-22 13:18:53,313] A new study created in memory with name: no-name-b71a222f-edb5-4e9c-ac64-a58604b04618
[I 2023-12-22 13:18:55,977] Trial 0 finished with value: inf and parameters: {'num_epochs': 5, 'batch_size': 8, 'learning_rate': 3.7627685371289524e-05}. Best is trial 0 with value: inf.


Trial 0 failed with CUDA out-of-memory error. Trying with different parameters.


[I 2023-12-22 13:18:57,956] Trial 1 finished with value: inf and parameters: {'num_epochs': 4, 'batch_size': 8, 'learning_rate': 0.00011286938942566083}. Best is trial 0 with value: inf.


Trial 1 failed with CUDA out-of-memory error. Trying with different parameters.


Step,Training Loss
1125,0.336700
2250,0.373200
3375,0.365800
4500,0.367400
5625,0.364200
6750,0.365200
7875,0.374000


[I 2023-12-22 15:22:33,254] Trial 2 finished with value: 0.3496590554714203 and parameters: {'num_epochs': 7, 'batch_size': 4, 'learning_rate': 1.034185040056526e-05}. Best is trial 2 with value: 0.3496590554714203.


In [ ]:
# import optuna
# from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
# import torch

# # Define your model, data_collator, train_tokenized_dataset, and test_tokenized_dataset beforehand

# def objective(trial):
#     num_epochs = trial.suggest_int('num_epochs', 3, 10)
#     batch_size = trial.suggest_categorical('batch_size', [4, 8, 16])
#     learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

#     output_dir = "flant5-large-answ"

#     training_args = Seq2SeqTrainingArguments(
#         output_dir=output_dir,
#         per_device_train_batch_size=batch_size,  # Use the batch_size from the trial
#         learning_rate=learning_rate,  # Use the learning_rate from the trial
#         num_train_epochs=num_epochs,
#         logging_dir=f"{output_dir}/logs",
#         logging_strategy="epoch",
#         save_strategy="epoch",
#         report_to="tensorboard",
#         push_to_hub=True
#     )

#     trainer = Seq2SeqTrainer(
#         model=model,  # Define 'model' before this point
#         args=training_args,
#         data_collator=data_collator,
#         train_dataset=train_tokenized_dataset,
#         eval_dataset=test_tokenized_dataset
#     )

#     try:
#         trainer.train()
#         result = trainer.evaluate()
#         return result['eval_loss']
#     except RuntimeError as e:
#         print(f"Trial {trial.number} failed with CUDA out-of-memory error. Trying with different parameters.")
#         torch.cuda.empty_cache()
#         return float('inf')


# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=5)

# best_params = study.best_params


[I 2023-12-21 14:15:52,216] A new study created in memory with name: no-name-61015fb3-5bbe-46d9-b649-052ca981690b
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1125,0.416800
2250,0.354500
3375,0.355700
4500,0.351600
5625,0.349100
6750,0.346200
7875,0.345500
9000,0.340400
10125,0.336300
11250,0.336400


[I 2023-12-21 17:08:40,071] Trial 0 finished with value: 0.345909059047699 and parameters: {'num_epochs': 10, 'batch_size': 4, 'learning_rate': 0.00027910008002364457}. Best is trial 0 with value: 0.345909059047699.
[I 2023-12-21 17:08:41,630] Trial 1 finished with value: inf and parameters: {'num_epochs': 8, 'batch_size': 16, 'learning_rate': 0.00011586929473822548}. Best is trial 0 with value: 0.345909059047699.


Trial 1 failed with CUDA out-of-memory error. Trying with different parameters.


Step,Training Loss
1125,0.346000
2250,0.339300
3375,0.332700
4500,0.329000
5625,0.321400
6750,0.312800
7875,0.312900
9000,0.299100


In [ ]:
output_dir="flant5-large-answ"
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=6,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="epoch",
    save_strategy="epoch",
    report_to="tensorboard",
    push_to_hub = True
)

In [ ]:
model.config.use_cache = False

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_tokenized_dataset,
    eval_dataset=test_tokenized_dataset
)

In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
750,0.456800


TrainOutput(global_step=750, training_loss=0.4567866617838542, metrics={'train_runtime': 1017.3059, 'train_samples_per_second': 4.422, 'train_steps_per_second': 0.737, 'total_flos': 5200879335505920.0, 'train_loss': 0.4567866617838542, 'epoch': 1.0})

In [ ]:
trainer.evaluate()


{'eval_loss': 0.34393391013145447,
 'eval_runtime': 51.0068,
 'eval_samples_per_second': 9.803,
 'eval_steps_per_second': 1.235}

In [ ]:
# peft_save_model_id="flant5-large-answ"
# trainer.model.save_pretrained(peft_save_model_id, push_to_hub=True)
# tokenizer.save_pretrained(peft_save_model_id, push_to_hub=True)
# trainer.model.base_model.save_pretrained(peft_save_model_id, push_to_hub=True)